In [1]:
from typing import Iterator, List, Union, Tuple
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Model
from tensorflow.python.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import MeanAbsoluteError, MeanAbsolutePercentageError, MeanSquaredError
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import History
import os



In [2]:
from tensorflow.keras import initializers


In [25]:
dataset = ('C:/Users/Ketan/Thesis/PilotNet/src/driving_dataset')

In [4]:
# Read the data
df = pd.io.parsers.read_csv(os.path.join(dataset, 'data_new.csv'))
# Split data into training and validation sets
df_train, df_val = train_test_split(df, test_size=.2)


In [5]:
df_train.mean(axis=0)

C:\Users\Ketan\AppData\Local\Temp/ipykernel_8212/2869931957.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_train.mean(axis=0)


angle    1.804542
dtype: float64

In [6]:
datagen=ImageDataGenerator(rescale=1./255)

In [7]:
train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory=dataset, x_col="image", y_col="angle", 
                                            class_mode="raw", target_size=(256, 455), batch_size=32)

Found 36324 validated image filenames.


In [8]:
val_generator=datagen.flow_from_dataframe(dataframe=df_val, directory=dataset, x_col="image", y_col="angle", 
                                            class_mode="raw", target_size=(256, 455), batch_size=32)

Found 9081 validated image filenames.


In [9]:
input_shape = (256, 455, 3)

In [10]:
from keras.layers import Flatten, Dense
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import pooling
from keras.layers import Dropout
from keras.layers import InputLayer



In [11]:
# define PilotNet model, with batch normalization included.
def get_model(input_shape):
    model = Sequential([
        InputLayer(input_shape=input_shape),
        BatchNormalization(),
        Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu', input_shape=input_shape, padding='valid',
               kernel_initializer=initializers.truncated_normal(stddev=0.01)),
#         BatchNormalization(),
#         pooling.MaxPooling2D(pool_size=(2, 2),padding='same'),
        Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu', padding='valid'),
#         BatchNormalization(axis=1),
#         pooling.MaxPooling2D(pool_size=(2, 2),padding='same'),

        Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu', padding='valid'),
#         BatchNormalization(axis=1),
#         pooling.MaxPooling2D(pool_size=(2, 2),padding='same'),

        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu', padding='valid'),
#         BatchNormalization(axis=1),
#         pooling.MaxPooling2D(pool_size=(2, 2),padding='same'),

        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu',name='target_layer', padding='valid'),
#         BatchNormalization(axis=1),
#         pooling.MaxPooling2D(pool_size=(2, 2),padding='same'),

        Flatten(),
        Dense(1164, activation='relu', kernel_regularizer= keras.regularizers.L2(l2=1e-3)),
        Dropout(.5),
        Dense(100, activation='relu', kernel_regularizer= keras.regularizers.L2(l2=1e-3)),
#         BatchNormalization(),
        Dropout(.5),
        Dense(50, activation='relu', kernel_regularizer= keras.regularizers.L2(l2=1e-3)),
#         BatchNormalization(),
        Dropout(.25),
        Dense(10, activation='relu', kernel_regularizer= keras.regularizers.L2(l2=1e-3)),
        Dropout(.25),

#         BatchNormalization(),
        Dense(1)
    ])
    
    return model

model = get_model(input_shape)
#sgd = SGD(learning_rate=1e-3, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-04), loss="mse",  metrics=[keras.metrics.MeanSquaredError()]) 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 256, 455, 3)       12        
_________________________________________________________________
conv2d (Conv2D)              (None, 126, 226, 24)      1824      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 111, 36)       21636     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 54, 48)        43248     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 27, 52, 64)        27712     
_________________________________________________________________
target_layer (Conv2D)        (None, 25, 50, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 80000)             0

In [20]:
model = Sequential()
model.add(Conv2D(16,3,3, input_shape=(256,455,3), activation='relu', kernel_initializer=initializers.truncated_normal(stddev=0.01)))
# model.add(pooling.MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(32, 3, 3, activation='relu'))
# model.add(pooling.MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(64, 3, 3, activation='relu'))
# model.add(pooling.MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(64, 3, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(960, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(500, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(20, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(1))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-04), loss='mse',metrics=['MeanSquaredError'])

In [ ]:
model.summary()

In [ ]:
def build_model(drop_prob = 0.2, learn_rate=0.001, constraint=maxnorm(3), reg = None, verbose = False): 
    backend.clear_session()
    model = Sequential()

    ###--- Convolutional Neural Network --- ### 
    input_shape = (180, 320, 3)

    model.add(Cropping2D(cropping=((60, 0), (0, 0)), input_shape=input_shape))
    # model.add(Lambda(resize, input_shape=input_shape))
    model.add(BatchNormalization(name='r0'))

    ### conv. layers 
    model.add(Convolution2D(filters=24,
                            kernel_size=(5, 5),
                            strides = (2, 2),
                            activation='relu',
                            kernel_regularizer=reg, 
                            input_shape=input_shape, 
                            ))
    '''READ THIS try adding max pool - before min loss was 7.04'''
#     model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization(name='r1'))

    model.add(Convolution2D(filters=36,
                            kernel_size=(5, 5),
                            strides = (2, 2),
                            activation='relu',
                            kernel_regularizer=reg, 
                            ))
    model.add(BatchNormalization(name='r2'))

    model.add(Convolution2D(filters=48,
                            kernel_size=(5, 5),
                            strides = (2, 2),
                            activation='relu',
                            kernel_regularizer=reg, 
                            ))
    model.add(BatchNormalization(name='r3'))

    model.add(Convolution2D(filters=64,
                            kernel_size=(3, 3),
                            strides = (1, 1),
                            activation='relu',
                            kernel_regularizer=reg, 
                            ))
    model.add(BatchNormalization(name='r4'))

    model.add(Convolution2D(filters=64,
                            kernel_size=(1, 1),
                            strides = (2, 2),
                            activation='relu',
                            kernel_regularizer=reg, 
                            ))
    model.add(BatchNormalization(name='r5'))

    ### fully connect layers 
    model.add(Flatten())
    model.add(Dense(1164, activation='relu', kernel_constraint=constraint))
    model.add(Dropout(drop_prob))

    model.add(Dense(100, activation='relu', kernel_constraint=constraint))
    model.add(Dropout(drop_prob))

    model.add(Dense(50, activation='relu', kernel_constraint=constraint))
    model.add(Dropout(drop_prob))

    model.add(Dense(10, activation='relu', kernel_constraint=constraint))
    model.add(Dropout(drop_prob))

    model.add(Dense(1, activation='linear'))

    if verbose: 
        # Summarize the model
        model.summary()
    
    #compile 
    model.compile(loss='mse', optimizer=optimizers.Adam(lr=learn_rate))

    return model

In [12]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

In [13]:
checkpoint_filepath = './logs2'


In [14]:
callback1 = keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', verbose=0, 
                                               save_best_only=False, save_weights_only=False, mode='auto', 
                                               save_freq='epoch',options=None)

In [15]:
callback2 = keras.callbacks.EarlyStopping(monitor="val_loss", patience = 2) 


In [ ]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10, callbacks=[callback1,callback2])

In [16]:
model.save('nvidia_regression_v1.h5')